### Data Augumentation - Augraphy

#### 1) 패키지/라이브러리 다운로드 및 임포트

In [1]:
# !pip install augraphy

In [2]:
import os

import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

#### 2) Augraphy - Default Pipeline
 - Random

In [3]:
# 원본 이미지 경로 탐색 및 파일명 추출
fpath_load = "../data/train/"
fname_list = os.listdir(fpath_load)
fname_list.sort()
print(len(fname_list))

# 증강 이미지 저장 경로 설정
fpath_save = "../data/train_augmented/"
if not os.path.exists(fpath_save):
    os.makedirs(fpath_save)

1570


In [4]:
from augraphy import *


# 이미지 증강
def augment_images(image_augmented_batch=10, image_augmented_iteration=5):
    # Make Pipeline for Augraphy
    pipeline = default_augraphy_pipeline()

    # Make Batches of Images
    image_list = []
    for i in range(image_augmented_batch):
        image_list.append(cv.imread(fpath_load + fname_list[i]))
        # 원본 이미지 저장
        cv.imwrite(fpath_save + fname_list[i], image_list[i])
        print(f"원본 이미지 저장: {fpath_save}{fname_list[i]}")

    # Augument Images
    for i in range(1, image_augmented_iteration + 1):

        # 이미지 증강
        image_augmented_list = pipeline(image_list)

        for j, image in enumerate(image_augmented_list):
            # 증강 이미지 저장
            fname = fname_list[j].split(".")[0]
            fname_save = f"{fpath_save}{fname}_{i}.jpg"
            cv.imwrite(fname_save, image)
            print(f"증강 이미지 저장: {fname_save}")

    return image_augmented_list

In [5]:
# image_augmented_list = augment_images(
#     image_augmented_batch=len(fname_list), image_augmented_iteration=10
# )

image_augmented_list = augment_images(
    image_augmented_batch=5, image_augmented_iteration=3
)

원본 이미지 저장: ../data/train_augmented/002f99746285dfdd.jpg
원본 이미지 저장: ../data/train_augmented/008ccd231e1fea5d.jpg
원본 이미지 저장: ../data/train_augmented/008f5911bfda7695.jpg
원본 이미지 저장: ../data/train_augmented/009235e4c9c07af5.jpg
원본 이미지 저장: ../data/train_augmented/00b2f44967580c74.jpg


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


증강 이미지 저장: ../data/train_augmented/002f99746285dfdd_1.jpg
증강 이미지 저장: ../data/train_augmented/008ccd231e1fea5d_1.jpg
증강 이미지 저장: ../data/train_augmented/008f5911bfda7695_1.jpg
증강 이미지 저장: ../data/train_augmented/009235e4c9c07af5_1.jpg
증강 이미지 저장: ../data/train_augmented/00b2f44967580c74_1.jpg
증강 이미지 저장: ../data/train_augmented/002f99746285dfdd_2.jpg
증강 이미지 저장: ../data/train_augmented/008ccd231e1fea5d_2.jpg
증강 이미지 저장: ../data/train_augmented/008f5911bfda7695_2.jpg
증강 이미지 저장: ../data/train_augmented/009235e4c9c07af5_2.jpg
증강 이미지 저장: ../data/train_augmented/00b2f44967580c74_2.jpg
증강 이미지 저장: ../data/train_augmented/002f99746285dfdd_3.jpg
증강 이미지 저장: ../data/train_augmented/008ccd231e1fea5d_3.jpg
증강 이미지 저장: ../data/train_augmented/008f5911bfda7695_3.jpg
증강 이미지 저장: ../data/train_augmented/009235e4c9c07af5_3.jpg
증강 이미지 저장: ../data/train_augmented/00b2f44967580c74_3.jpg


#### 3) Augraphy - User Defined Pipeline